In [1]:
import pandas as pd

# Načítanie dát zo súborov
salaries_and_inflation_df = pd.read_csv("Inflace_a_platy_FACT.csv", encoding="utf-8")
property_df = pd.read_csv("Brno_vycistene_udaje.csv", encoding="utf-8")


In [2]:

# Filtrovanie len záznamov s typom ponuky 'rent'
property_df = property_df[property_df["typ_nabidky"].str.lower() == "rent"]

# Spojenie tabuliek na základe stĺpca 'rok'
merged_df = pd.merge(property_df, salaries_and_inflation_df[['rok', 'platy_muzi', 'platy_zeny']], on='rok', how='left')


In [3]:
# Konverzia stĺpcov platy_muzi a platy_zeny na číselný typ
merged_df['platy_muzi'] = pd.to_numeric(merged_df['platy_muzi'].str.replace(' ', '').str.replace(',', '.'), errors='coerce')
merged_df['platy_zeny'] = pd.to_numeric(merged_df['platy_zeny'].str.replace(' ', '').str.replace(',', '.'), errors='coerce')

# Predpoklad maximálnej mesačnej splátky (40% platu)
merged_df['max_splatka_muzi'] = merged_df['platy_muzi'] * 0.40
merged_df['max_splatka_zeny'] = merged_df['platy_zeny'] * 0.40
merged_df['max_splatka_par'] = (merged_df['platy_muzi'] + merged_df['platy_zeny']) * 0.40

# Výpočet pomeru mesačných splátok k platu
merged_df["splatka_k_platu_muzi"] = merged_df["cena"] / merged_df["max_splatka_muzi"]
merged_df["splatka_k_platu_zeny"] = merged_df["cena"] / merged_df["max_splatka_zeny"]
merged_df["splatka_k_platu_par"] = merged_df["cena"] / merged_df["max_splatka_par"]

# Pridanie stĺpcov s hodnotami TRUE/FALSE
merged_df["muzi_si_mohou_dovolit"] = merged_df["splatka_k_platu_muzi"] <= 1
merged_df["zeny_si_mohou_dovolit"] = merged_df["splatka_k_platu_zeny"] <= 1
merged_df["par_si_muze_dovolit"] = merged_df["splatka_k_platu_par"] <= 1

In [5]:
# Výpočet priemerných platov a maximálnych splátok pre každý rok, vrátane dostupnosti splácania
years = merged_df["rok"].unique()
results = []

for year in years:
    yearly_data = merged_df[merged_df["rok"] == year]
    average_salary_male = yearly_data["platy_muzi"].mean()
    average_salary_female = yearly_data["platy_zeny"].mean()
    max_payment_male = average_salary_male * 0.40
    max_payment_female = average_salary_female * 0.40
    max_payment_couple = (average_salary_male + average_salary_female) * 0.40
    
    can_afford_male = yearly_data["muzi_si_mohou_dovolit"].all()
    can_afford_female = yearly_data["zeny_si_mohou_dovolit"].all()
    can_afford_couple = yearly_data["par_si_muze_dovolit"].all()
    
    results.append({
        "rok": year,
        "prumerny_plat_muzi": average_salary_male,
        "prumerny_plat_zeny": average_salary_female,
        "max_splatka_muzi": max_payment_male,
        "max_splatka_zeny": max_payment_female,
        "max_splatka_par": max_payment_couple,
        "muzi_si_mohou_dovolit": can_afford_male,
        "zeny_si_mohou_dovolit": can_afford_female,
        "par_si_muze_dovolit": can_afford_couple
    })

results_df = pd.DataFrame(results)
print("Results for each year:")
print(results_df)

# Pridanie stĺpcov "cena", "nasetrene_10", "cena_90", "mesicni_splatka_90", "mena", "dipozice", "uzitna_plocha", "patro", "typ_nemovitosti" a "oblast" do výsledkového dataframe
results_df = merged_df[['rok', 'cena', 'mena', 'forma_platby','max_splatka_muzi', 'max_splatka_zeny', 'max_splatka_par', 'muzi_si_mohou_dovolit', 'zeny_si_mohou_dovolit', 'par_si_muze_dovolit', 'dipozice', 'uzitna_plocha', 'typ_nemovitosti']]


# Uloženie výsledkov do CSV
results_df.to_csv("Pronajmy_FACT.csv", index=False, encoding="utf-8")
print("Results for each year saved successfully")

# Uloženie konečného dataframe so všetkými výpočtami
merged_df.to_csv("Brno_pronajmy_vysledky_combined.csv", index=False, encoding="utf-8")
print("Combined results saved successfully")

Results for each year:
    rok  prumerny_plat_muzi  prumerny_plat_zeny  max_splatka_muzi  \
0  2022             56206.0             40235.0           22482.4   
1  2021             50996.0             37591.0           20398.4   
2  2020             46538.0             34250.0           18615.2   
3  2023             58993.0             42375.0           23597.2   
4  2019             44392.0             32733.0           17756.8   
5  2018             39906.0             29942.0           15962.4   
6  2016             33518.0             24927.0           13407.2   
7  2024                 NaN                 NaN               NaN   
8  2017             36800.0             27255.0           14720.0   

   max_splatka_zeny  max_splatka_par  muzi_si_mohou_dovolit  \
0           16094.0          38576.4                  False   
1           15036.4          35434.8                  False   
2           13700.0          32315.2                  False   
3           16950.0          40547